Process CLEAN

In [19]:
data = pd.read_csv('/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/CLEAN.csv')
# Identify columns with only a single unique value
single_value_columns = data.columns[data.nunique() == 1].tolist()
print("Columns with a single unique value: ", single_value_columns)
for col in single_value_columns:
    print(f"Unique value in {col}: {data[col].unique()[0]}")

data = data.drop(single_value_columns, axis=1)



Columns with a single unique value:  ['assetClass', 'blockTradeIndicator', 'customBasketIndicator', 'deliveryType', 'disseminationTimestamp', 'event', 'instrumentType', 'leg1NotionalScheduleType', 'leg2NotionalScheduleType', 'nonStandardTermIndicator', 'packageIndicator', 'postPricedSwapIndicator', 'primeBrokerageTransactionIndicator']
Unique value in assetClass: IR
Unique value in blockTradeIndicator: nan
Unique value in customBasketIndicator: False
Unique value in deliveryType: CASH
Unique value in disseminationTimestamp: disseminationTimestamp
Unique value in event: TRAD
Unique value in instrumentType: Swap
Unique value in leg1NotionalScheduleType: Constant
Unique value in leg2NotionalScheduleType: Constant
Unique value in nonStandardTermIndicator: nan
Unique value in packageIndicator: False
Unique value in postPricedSwapIndicator: False
Unique value in primeBrokerageTransactionIndicator: nan


PROCESS CLEAN DATA AND SAVE THE SCALER AS robust_scaler.pkl

In [88]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import joblib

your_path = '/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/'
training_data_name = 'CLEAN'

# Load the data
data = pd.read_csv('/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/CLEAN.csv')

# Drop 'disseminationTimestamp' and its derived columns
data = data.drop(columns=['disseminationTimestamp','sDRreceiptTimestamp'])

# Step 1: Handle Missing Data
data_filled = data.fillna(0)

# Step 2: Extract Information from DateTime Columns
datetime_columns = ['effectiveDate', 'eventDateTime', 'executionDateTime', 'expirationDate']

def extract_date_features(df, column):
    df[column] = pd.to_datetime(df[column], errors='coerce')
    df[column + '_year'] = df[column].dt.year
    df[column + '_month'] = df[column].dt.month
    df[column + '_day'] = df[column].dt.day
    df[column + '_hour'] = df[column].dt.hour
    df[column + '_minute'] = df[column].dt.minute
    df[column + '_second'] = df[column].dt.second
    df[column + '_weekday'] = df[column].dt.weekday
    # Drop the original datetime column
    df = df.drop(column, axis=1)
    return df

for col in datetime_columns:
    data_filled = extract_date_features(data_filled, col)

original_dtypes = data.dtypes.to_dict()

# Convert boolean columns to binary (1/0) before one-hot encoding other categorical columns
data_filled = data_filled*1



# Step 3: Encode Categorical Variables
categorical_columns = data_filled.select_dtypes(include=['object', 'bool']).columns.tolist()
categorical_columns = [col for col in categorical_columns if col not in datetime_columns]
data_encoded = pd.get_dummies(data_filled, columns=categorical_columns, drop_first=True)

# Step 3.5: Identify and Drop Single-Value Columns
single_value_columns = data_encoded.columns[data_encoded.nunique() == 1].tolist()

# Optionally print these columns and their unique values
print("Columns with a single unique value: ", single_value_columns)

# Drop the single-value columns
data_encoded = data_encoded.drop(columns=single_value_columns)

# Define numerical_columns here, after all column dropping and adding has occurred
numerical_columns = data_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Check for NaN or infinite values in numerical columns before scaling
nan_inf_columns = data_encoded[numerical_columns].columns[data_encoded[numerical_columns].isna().any() | np.isinf(data_encoded[numerical_columns]).any()].tolist()
print("Columns with NaN or infinite values before scaling: ", nan_inf_columns)

# Step 4: Scale/Normalize Data
numerical_columns = data_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()
scaler = RobustScaler()
data_encoded[numerical_columns] = scaler.fit_transform(data_encoded[numerical_columns])

# Save the scaler for future use
scaler_filename = 'robust_scaler.pkl'
joblib.dump(scaler, scaler_filename)

# Your preprocessed data is now stored in `data_encoded`, and your scaler is saved as `robust_scaler.pkl`

# Save the processed data to a CSV file in this environment
data_encoded.to_csv('/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/CLEAN_Scaled.csv', index=False)

train_encoded_columns = data_encoded.columns


Columns with a single unique value:  ['customBasketIndicator', 'packageIndicator', 'postPricedSwapIndicator', 'effectiveDate_hour', 'effectiveDate_minute', 'effectiveDate_second', 'expirationDate_hour', 'expirationDate_minute', 'expirationDate_second']
Columns with NaN or infinite values before scaling:  []


In [100]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import joblib

# Load the data
t_data = pd.read_csv('/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/test_Data.csv')

# Drop 'disseminationTimestamp' and its derived columns
t_data = t_data.drop(columns=['disseminationTimestamp','sDRreceiptTimestamp'])

# Step 1: Handle Missing Data
t_data_filled = t_data.fillna(0)

# Step 2: Extract Information from DateTime Columns
t_datetime_columns = ['effectiveDate', 'eventDateTime', 'executionDateTime', 'expirationDate']

def extract_date_features(df, column):
    df[column] = pd.to_datetime(df[column], errors='coerce')
    df[column + '_year'] = df[column].dt.year
    df[column + '_month'] = df[column].dt.month
    df[column + '_day'] = df[column].dt.day
    df[column + '_hour'] = df[column].dt.hour
    df[column + '_minute'] = df[column].dt.minute
    df[column + '_second'] = df[column].dt.second
    df[column + '_weekday'] = df[column].dt.weekday
    # Drop the original datetime column
    df = df.drop(column, axis=1)
    return df

for col in t_datetime_columns:
    t_data_filled = extract_date_features(t_data_filled, col)

# Convert boolean columns to binary (1/0) before one-hot encoding other categorical columns
t_data_filled = t_data_filled*1

# Step 3: Encode Categorical Variables
t_categorical_columns = t_data_filled.select_dtypes(include=['object', 'bool']).columns.tolist()
t_categorical_columns = [col for col in t_categorical_columns if col not in t_datetime_columns]
t_data_encoded = pd.get_dummies(t_data_filled, columns=t_categorical_columns, drop_first=True)

# Step 3.5: Identify and Drop Single-Value Columns
t_single_value_columns = t_data_encoded.columns[t_data_encoded.nunique() == 1].tolist()

# Optionally print these columns and their unique values
print("Columns with a single unique value: ", t_single_value_columns)

# Drop the single-value columns
t_data_encoded = t_data_encoded.drop(columns=t_single_value_columns)

# Define numerical_columns here, after all column dropping and adding has occurred
t_numerical_columns = t_data_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Check for NaN or infinite values in numerical columns before scaling
t_nan_inf_columns = t_data_encoded[t_numerical_columns].columns[t_data_encoded[t_numerical_columns].isna().any() | np.isinf(t_data_encoded[t_numerical_columns]).any()].tolist()
print("Columns with NaN or infinite values before scaling: ", t_nan_inf_columns)

def match_columns(new_data, train_encoded_columns, original_dtypes):
    missing_cols = {col: 0 for col in train_encoded_columns if col not in new_data.columns}
    extra_cols = [col for col in new_data.columns if col not in train_encoded_columns]
    
    # Add missing columns with 0s
    new_data = pd.concat([new_data, pd.DataFrame(missing_cols, index=new_data.index)], axis=1)
    
    # Remove extra columns
    new_data = new_data.drop(columns=extra_cols)
    
    # Ensure columns are in the same order as training data
    new_data = new_data[train_encoded_columns]
    
    # Convert columns back to their original type
    for col, original_dtype in original_dtypes.items():
        if col in new_data.columns:  # Ensure the column exists in new_data
            if pd.api.types.is_categorical_dtype(original_dtype):
                new_data[col] = pd.Categorical(new_data[col])
            else:
                new_data[col] = new_data[col].astype(original_dtype)
            
            # Debug: Print the column, its intended dtype, and the resulting dtype
            print(f"{col}: intended dtype: {original_dtype}, converted dtype: {new_data[col].dtype}")
    
    return new_data



# Apply the function
t_data_matched = match_columns(t_data_encoded, train_encoded_columns, original_dtypes)

# Check if all columns in t_data_matched are equal to train_encoded_columns
if list(t_data_matched.columns) == list(train_encoded_columns):

    print("All columns in t_data_matched are equal to train_encoded_columns.")
else:
    print("Columns in t_data_matched and train_encoded_columns do not match.")

    # Check if the columns are the same but in a different order
    if set(t_data_matched.columns) == set(train_encoded_columns):
        print("The datasets have the same columns, but they are in a different order.")
        
        # You can reorder the columns in t_data_matched to match train_encoded_columns
        t_data_matched = t_data_matched[train_encoded_columns]
        print("Columns in t_data_matched reordered to match train_encoded_columns.")
    else:
        # Identify and print the mismatched columns
        extra_cols = set(t_data_matched.columns) - set(train_encoded_columns)
        missing_cols = set(train_encoded_columns) - set(t_data_matched.columns)
        print("Extra columns in t_data_matched: ", extra_cols)
        print("Missing columns in t_data_matched: ", missing_cols)


import joblib

# Load the saved scaler
scaler = joblib.load('/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/robust_scaler.pkl')
print(numerical_columns)
# Scale the numerical columns of t_data_matched using the 'numerical_columns' from the training data
t_data_matched[t_numerical_columns] = scaler.transform(t_data_matched[t_numerical_columns])

t_data_encoded.to_csv('/Users/elliotlindestam/Documents/Skola/Indek icloud/trioptima/test_Scaled.csv', index=False)

Columns with a single unique value:  ['blockTradeIndicator', 'customBasketIndicator', 'nonStandardTermIndicator', 'packageIndicator', 'platformID', 'postPricedSwapIndicator', 'primeBrokerageTransactionIndicator', 'effectiveDate_hour', 'effectiveDate_minute', 'effectiveDate_second', 'eventDateTime_year', 'eventDateTime_month', 'eventDateTime_day', 'eventDateTime_weekday', 'executionDateTime_year', 'executionDateTime_month', 'executionDateTime_day', 'executionDateTime_weekday', 'expirationDate_hour', 'expirationDate_minute', 'expirationDate_second']
Columns with NaN or infinite values before scaling:  []
leg1NotionalAmount: intended dtype: float64, converted dtype: float64
leg2NotionalAmount: intended dtype: float64, converted dtype: float64
leg2ResetFrequencyMultiplier: intended dtype: float64, converted dtype: float64
leg1FixedRate: intended dtype: float64, converted dtype: float64
leg1FixedRatePaymentFrequencyMultiplier: intended dtype: float64, converted dtype: float64
leg2FloatingRa

/var/folders/rq/30rpw85n53d2zxnt_c5qnrx40000gn/T/ipykernel_40751/4098890374.py:74: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(original_dtype):
